In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('precision',2)

In [3]:
data_path = '../ml-latest'
all_files = [i for i in os.listdir(data_path) if i.endswith('.csv')]
print(all_files)

['genome-tags.csv', 'movies.csv', 'genome-scores.csv', 'tags.csv', 'ratings.csv', 'movies_clean.csv', 'links.csv']


In [4]:
def get_data(fn):
    df = pd.read_csv(os.path.join(data_path, '{}.csv'.format(fn)))
    print('{} total: {} rows.'.format(fn, len(df)))
    return df

In [5]:
movies = get_data('movies_clean')#.head(1000)
movies.head()

movies_clean total: 45843 rows.


,Unnamed: 0,movieId,title,genres,year,Action,Fantasy,Children,Musical,Documentary,...,Horror,Film-Noir,Sci-Fi,Comedy,Drama,IMAX,War,Crime,Animation,Western
0,0,1,Toy Story,[0 1 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0],1995,0,1,1,0,0,...,0,0,0,1,0,0,0,0,1,0
1,1,2,Jumanji,[0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0],1995,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,3,Grumpier Old Men,[0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0],1995,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,3,4,Waiting to Exhale,[0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0],1995,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
4,4,5,Father of the Bride Part II,[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0],1995,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [6]:
n_movies = len(movies)

In [ ]:
tags = [i for i in movies.columns if i not in ['Unnamed:0', 'movieId', 'title', 'genres', 'year']]

In [ ]:
# generate movie similarity matrix using cosine distance
# rows are movies and columns are tags
from sklearn.metrics.pairwise import pairwise_distances
similarity_matrix = pairwise_distances(movies[tags].as_matrix(), metric='cosine')

In [ ]:
def get_similar_movie(movie_id):
    movie_idx = movie_id - 1
    row = similarity_matrix[movie_idx, :]
    most_similar_idx = np.argmax(row)
    most_similar_score = row[most_similar_idx]
    most_similar_movie = movies[movies.movieId==most_similar_idx + 1].title.iloc[0]
    print('If you liked {}, we recommend {} with score {}.'.format(movies[movies.movieId==movie_id].title.iloc[0], most_similar_movie, most_similar_score))

In [ ]:
get_similar_movie(1)